In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/machine-translate/Hindi_English_Truncated_Corpus.csv


# 1. Introduction

## Name :Kumar Rohit <br>
## PRN : 21070126052 <br>
## Batch : AIML - A3 <br>

# 1.2. Importing the Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding , LSTM , Dense, RepeatVector , TimeDistributed , Input
from tensorflow.keras.models import Model
from tensorflow.keras.losses import sparse_categorical_crossentropy
import re
import string
from string import digits
import numpy as np

# 1.3. Importing the Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/machine-translate/Hindi_English_Truncated_Corpus.csv')
df['source'].value_counts()

source
tides        50000
ted          39881
indic2012    37726
Name: count, dtype: int64

In [ ]:
df = df[(df.english_sentence.apply(lambda x: len(str(x))<=30)) &
            (df.hindi_sentence.apply(lambda x: len(str(x))<=30))]

# 2. Data Preprocessing

# 2.1. Changing the Uppercase to Lowercase

In [ ]:
# Changing uppercase to lowercase
df['english_sentence'] = df['english_sentence'].apply(lambda x:str(x).lower())
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: x.lower())

# Remove quotes
df['english_sentence'] = df['english_sentence'].apply(lambda x:re.sub("'",'',x))
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: re.sub("'",'',x))

# 2.2. Removing the Punctuations

In [ ]:
to_exclude = set(string.punctuation) # Set of all special characters
print("Punctuations to be excluded : ",to_exclude)

# Remove all the special characters
df['english_sentence'] = df['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in to_exclude))
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in to_exclude))

Punctuations to be excluded :  {'<', '/', '.', '(', '$', ';', '~', '_', ':', '?', '\\', ',', '+', '^', '"', '>', '{', '*', ')', '}', '@', '[', '!', '&', '-', ']', '|', '%', '`', '#', "'", '='}


# 2.3. Removing the Digits

In [ ]:
# Remove all the digits
from string import digits
remove_digits = str.maketrans('', '', digits)
df['english_sentence'] = df['english_sentence'].apply(lambda x: x.translate(remove_digits))
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
df

,source,english_sentence,hindi_sentence
11,indic2012,category religious text,श्रेणीधर्मग्रन्थ
23,ted,this changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced,उत्पन्न नहीं कि जाती थी
33,indic2012,maine,मेन
35,ted,can you imagine saying that,क्या आप ये कल्पना कर सकते है
...,...,...,...
127580,indic2012,google author,गूगल अर्थ
127581,tides,the die was cast,सांचा ढल गया
127582,indic2012,devotional period,भक्ति काल १३७५१७००
127590,indic2012,travelling,यात्राएं


In [ ]:
# Remove all the digits
from string import digits
remove_digits = str.maketrans('', '', digits)
df['english_sentence'] = df['english_sentence'].apply(lambda x: x.translate(remove_digits))
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
df

,source,english_sentence,hindi_sentence
11,indic2012,category religious text,श्रेणीधर्मग्रन्थ
23,ted,this changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced,उत्पन्न नहीं कि जाती थी
33,indic2012,maine,मेन
35,ted,can you imagine saying that,क्या आप ये कल्पना कर सकते है
...,...,...,...
127580,indic2012,google author,गूगल अर्थ
127581,tides,the die was cast,सांचा ढल गया
127582,indic2012,devotional period,भक्ति काल
127590,indic2012,travelling,यात्राएं


# 2.4. Removing the Extra Spaces

In [ ]:
# Remove extra spaces
df['english_sentence'] = df['english_sentence'].apply(lambda x: x.strip())
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: x.strip())
df['english_sentence'] = df['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))
df

,source,english_sentence,hindi_sentence
11,indic2012,category religious text,श्रेणीधर्मग्रन्थ
23,ted,this changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced,उत्पन्न नहीं कि जाती थी
33,indic2012,maine,मेन
35,ted,can you imagine saying that,क्या आप ये कल्पना कर सकते है
...,...,...,...
127580,indic2012,google author,गूगल अर्थ
127581,tides,the die was cast,सांचा ढल गया
127582,indic2012,devotional period,भक्ति काल
127590,indic2012,travelling,यात्राएं


# 2.5. Adding the Start and End Sequence

In [ ]:
# Create 2 list to send the data
input_text = []
target_text = []

# Creating the characters
input_characters = set()
target_characters = set()

for eng, hin in df[['english_sentence','hindi_sentence']].itertuples(index=False):
    target = 'START_ ' + hin + ' _END' # End Sequence
    input_text.append(eng)
    target_text.append(target)

    for char in eng.split():
        if char not in input_characters:
            input_characters.add(char)

    for hin_char in hin.split():
        if hin_char not in target_characters:
            target_characters.add(hin_char)

# 2.6. Printing the Data

In [ ]:
# Making the input_char
input_char = sorted(list(input_characters))
target_char = sorted(list(target_characters))

# Encoder definining
num_encoder_tokens = len(input_char)
num_decoder_tokens = len(target_char) + 1

max_encoder_seq_length = max([len(txt) for txt in input_text])
max_decoder_seq_length = max([len(txt) for txt in target_text])
print(num_encoder_tokens)
print(num_decoder_tokens)
print(max_encoder_seq_length)
print(max_decoder_seq_length)

9232
8666
30
42


In [ ]:
# Printing the data
print("Number of samples:", len(input_text))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 18416
Number of unique input tokens: 9232
Number of unique output tokens: 8666
Max sequence length for inputs: 30
Max sequence length for outputs: 42


# 3. Model Building

# 3.1. Creating the Dictionary

In [ ]:
# Creating the dictionary
input_token_index = dict([(word,i+1)for i, word in enumerate(input_char)])
target_token_index = dict([(word,i+1)for i, word in enumerate(target_char)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

# 3.2. Creating the Batch

In [ ]:
# Splitting the data
from sklearn.model_selection import train_test_split
X,y = df.english_sentence , df.hindi_sentence
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [ ]:
# Printinng the shape
print(X_train.shape)
print(X_test.shape)

(16574,)
(1842,)


In [ ]:
# Defining the batch
def generate_batch(X,y,batch_size):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_encoder_seq_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_decoder_seq_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_decoder_seq_length, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# 3.3. Making the Encoder

In [ ]:
latent_dim = 50

In [ ]:
# Making the Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)

# Adding two more LSTM layers
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs1, state_h1, state_c1 = encoder_lstm1(enc_emb)
encoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs2, state_h2, state_c2 = encoder_lstm2(encoder_outputs1)
encoder_lstm3 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs3, state_h3, state_c3 = encoder_lstm3(encoder_outputs2)

encoder_states = [state_h3, state_c3]

# 3.4. Making the Decoder

In [ ]:
# Making the decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)

# Adding two more LSTM layers
decoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs1, _, _ = decoder_lstm1(dec_emb, initial_state=encoder_states)
decoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs2, _, _ = decoder_lstm2(decoder_outputs1)
decoder_lstm3 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs3, _, _ = decoder_lstm3(decoder_outputs2)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs3)


# 3.5. Making the Model

In [ ]:
# Making the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc']) # For classification we use adam and for regression we use rmsprop

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 50)     461600      ['input_3[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, None, 50),   20200       ['embedding[0][0]']              
                                 (None, 50),                                                      
                                 (None, 50)]                                                      
                                                                                              

# 4. Model Training

# 4.1. Training the Model

In [ ]:
# Training the model
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 512
epochs = 100

# 4.2. Fitting the Model

In [ ]:
# Fitting the model

model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/tmp/ipykernel_28/1164885454.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/100
32/32 [==============================] - 100s 2s/step - loss: 9.0152 - acc: 0.0424 - val_loss: 8.7644 - val_acc: 0.0434
Epoch 2/100
32/32 [==============================] - 56s 2s/step - loss: 7.8140 - acc: 0.0449 - val_loss: 7.2922 - val_acc: 0.0434
Epoch 3/100
32/32 [==============================] - 56s 2s/step - loss: 7.0481 - acc: 0.0449 - val_loss: 7.2119 - val_acc: 0.0434
Epoch 4/100
32/32 [==============================] - 54s 2s/step - loss: 6.9833 - acc: 0.0449 - val_loss: 7.2363 - val_acc: 0.0434
Epoch 5/100
32/32 [==============================] - 56s 2s/step - loss: 6.9641 - acc: 0.0449 - val_loss: 7.2586 - val_acc: 0.0434
Epoch 6/100
32/32 [==============================] - 56s 2s/step - loss: 6.9525 - acc: 0.0447 - val_loss: 7.2800 - val_acc: 0.0434
Epoch 7/100
32/32 [==============================] - 55s 2s/step - loss: 6.9420 - acc: 0.0447 - val_loss: 7.2986 - val_acc: 0.0434
Epoch 8/100
32/32 [==============================] - 56s 2s/step - loss: 6.9329 - 

# 5. Model Evaluation

# 5.1. Printing the Validation Score

In [ ]:
# Printing the validation score
model.evaluate_generator(generator = generate_batch(X_test, y_test, batch_size = batch_size), steps = val_samples//batch_size)

/tmp/ipykernel_28/2723858562.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(generator = generate_batch(X_test, y_test, batch_size = batch_size), steps = val_samples//batch_size)


[6.74385404586792, 0.11486103385686874]

# 5.2. Saving the Weights

In [ ]:
# Saving the weights
model.save_weights('nmt_eng_hin_translation.h5')

# 6. Model Prediction

# 6.1. Making the Inference Model

In [ ]:
# Inference

encoder_model = Model(encoder_inputs,encoder_states)

# 6.2. Making the Decoder Model

In [ ]:
# Decoder Setup
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [ ]:
dec_emb2 = dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence
decoder_outputs2, state_h2, state_c2 = decoder_lstm3(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0] = target_token_index['START_']
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == ' _END' or len(decoded_sentence) > 25):
            stop_condition = True
            # Update the target sequence (of length 1).
            target_seq = np.zeros((1,1))
            target_seq[0, 0] = sampled_token_index
            # Update states
            states_value = [h, c]
    return decoded_sentence

# 6.3. Printing the Predicted Output

In [ ]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
k+=2
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0])
print('Predicted Hindi Translation:', decoded_sentence[:])

1/1 [==============================] - 0s 22ms/step
Input English sentence: class west bengal
Actual Hindi Translation: श्रेणीपश्चिम बंगाल
Predicted Hindi Translation:  झंडा झंडा झंडा झंडा झंडा झंडा
